In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [2]:
csv_file = "2017.csv"
dog_df = pd.read_csv(csv_file)
dog_df.head()

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate
0,Dog Individual Spayed Female,BICHON FRISE,WHITE,CHLOE,15090,2017,12/15/2016 9:58
1,Dog Senior Citizen or Disability Spayed Female,CHIHUAHUA,WHITE/BLACK/BROWN,CHALUPA,15120,2017,3/23/2017 14:32
2,Dog Individual Spayed Female,LAB MIX,YELLOW,MIA,15133,2017,2/28/2017 10:59
3,Dog Individual Male,DACHSHUND,BLACK/BROWN/GREY,CODY,15108,2017,2/9/2017 13:40
4,Dog Individual Spayed Female,DACHSHUND,BLACK,SOPHIE,15108,2017,2/9/2017 13:40


In [3]:
csv_file = "AKC.csv"
AKC_df = pd.read_csv(csv_file)
AKC_df.head()

,Breed,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
0,Akita,26,28,80,120
1,Anatolian Sheepdog,27,29,100,150
2,Bernese Mountain Dog,23,27,85,110
3,Rottweiler,22,27,90,110
4,Saint Bernard,25,28,110,190


In [4]:
intelligence= "dog_intelligence.csv"
Intelligence_df = pd.read_csv(intelligence)
Intelligence_df.head()

,Breed,Classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,95%,1,4
1,Poodle,Brightest Dogs,95%,1,4
2,German Shepherd,Brightest Dogs,95%,1,4
3,Golden Retriever,Brightest Dogs,95%,1,4
4,Doberman Pinscher,Brightest Dogs,95%,1,4


### Create new data with select columns

In [5]:
new_dog_df = dog_df[['Breed', 'Color', 'DogName']].copy()
new_dog_df.head()

,Breed,Color,DogName
0,BICHON FRISE,WHITE,CHLOE
1,CHIHUAHUA,WHITE/BLACK/BROWN,CHALUPA
2,LAB MIX,YELLOW,MIA
3,DACHSHUND,BLACK/BROWN/GREY,CODY
4,DACHSHUND,BLACK,SOPHIE


In [6]:
New_AKC_df = AKC_df[['Breed', 'height_low_inches', 'height_high_inches']].copy()
New_AKC_df.head()

,Breed,height_low_inches,height_high_inches
0,Akita,26,28
1,Anatolian Sheepdog,27,29
2,Bernese Mountain Dog,23,27
3,Rottweiler,22,27
4,Saint Bernard,25,28


In [7]:
New_intelligence_df = Intelligence_df[["Breed", "Classification", "obey"]].copy()
New_intelligence_df.dtypes

Breed             object
Classification    object
obey              object
dtype: object

### Transform Dog DataFrame

In [8]:
# Rename the column headers
new_dog_df = new_dog_df.rename(columns={"Breed": "dog_breed", "Color": "dog_color", "DogName": "dog_name"})

new_dog_df.head()

,dog_breed,dog_color,dog_name
0,BICHON FRISE,WHITE,CHLOE
1,CHIHUAHUA,WHITE/BLACK/BROWN,CHALUPA
2,LAB MIX,YELLOW,MIA
3,DACHSHUND,BLACK/BROWN/GREY,CODY
4,DACHSHUND,BLACK,SOPHIE


In [9]:
new_intelligence_df = New_intelligence_df.rename(columns={"obey":"Obedience"})
#new_intelligence_df.set_index("Breed", inplace = True)
new_intelligence_df.head()

,Breed,Classification,Obedience
0,Border Collie,Brightest Dogs,95%
1,Poodle,Brightest Dogs,95%
2,German Shepherd,Brightest Dogs,95%
3,Golden Retriever,Brightest Dogs,95%
4,Doberman Pinscher,Brightest Dogs,95%


### Create database connection

In [10]:
connection_string = "postgres:postgres@localhost:5432/DogDB"
engine = create_engine(f'postgresql://{connection_string}')

### Load DataFrames into database

In [11]:
new_dog_df.to_sql(name='dog_2017', con=engine, if_exists='append', index=True)

In [12]:
New_AKC_df.to_sql(name='Breed', con=engine, if_exists='append', index=False)

In [13]:
new_intelligence_df.to_sql(name='Intelligence', con=engine, if_exists='append', index=True)

### Confirm data has been added by querying the data tables

In [14]:
pd.read_sql_query('select * from dog_2017', con=engine).head()

,index,dog_breed,dog_color,dog_name
0,0,BICHON FRISE,WHITE,CHLOE
1,1,CHIHUAHUA,WHITE/BLACK/BROWN,CHALUPA
2,2,LAB MIX,YELLOW,MIA
3,3,DACHSHUND,BLACK/BROWN/GREY,CODY
4,4,DACHSHUND,BLACK,SOPHIE


In [15]:
pd.read_sql_query('select * from "Breed"', con=engine).head()

,Breed,height_low_inches,height_high_inches
0,Akita,26,28
1,Anatolian Sheepdog,27,29
2,Bernese Mountain Dog,23,27
3,Rottweiler,22,27
4,Saint Bernard,25,28


In [16]:
pd.read_sql_query('select * from "Intelligence"', con=engine).head()

,index,Breed,Classification,Obedience
0,0,Border Collie,Brightest Dogs,95%
1,1,Poodle,Brightest Dogs,95%
2,2,German Shepherd,Brightest Dogs,95%
3,3,Golden Retriever,Brightest Dogs,95%
4,4,Doberman Pinscher,Brightest Dogs,95%
